# Задание 1
Импортируйте библиотеки pandas и numpy.
Загрузите "Boston House Prices dataset" из встроенных наборов данных библиотеки sklearn. Создайте датафреймы X и y из этих данных.

Разбейте эти датафреймы на тренировочные (X_train, y_train) и тестовые (X_test, y_test) с помощью функции train_test_split так, чтобы размер тестовой выборки
составлял 30% от всех данных, при этом аргумент random_state должен быть равен 42.
Создайте модель линейной регрессии под названием lr с помощью класса LinearRegression из модуля sklearn.linear_model.

Обучите модель на тренировочных данных (используйте все признаки) и сделайте предсказание на тестовых.
Вычислите R2 полученных предказаний с помощью r2_score из модуля sklearn.metrics.


In [3]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

boston = load_boston()
feature_names = boston["feature_names"]
X = pd.DataFrame(boston["data"], columns=feature_names)
y = pd.DataFrame(boston["target"], columns=["price"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [17]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test["price"], y_pred.flatten())
r2

0.7112260057484974

# Задание 2
Создайте модель под названием model с помощью RandomForestRegressor из модуля sklearn.ensemble.

Сделайте агрумент n_estimators равным 1000,
max_depth должен быть равен 12 и random_state сделайте равным 42.

Обучите модель на тренировочных данных аналогично тому, как вы обучали модель LinearRegression,
но при этом в метод fit вместо датафрейма y_train поставьте y_train.values[:, 0],
чтобы получить из датафрейма одномерный массив Numpy,
так как для класса RandomForestRegressor в данном методе для аргумента y предпочтительно применение массивов вместо датафрейма.

Сделайте предсказание на тестовых данных и посчитайте R2. Сравните с результатом из предыдущего задания.
Напишите в комментариях к коду, какая модель в данном случае работает лучше.


In [18]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 1000, max_depth = 12, random_state = 42)
model.fit(X_train, y_train.values[:, 0])
y_pred_md = model.predict(X_test)

r2_model = r2_score(y_test["price"], y_pred_md.flatten())
r2_model

0.87472606157312

In [ ]:
# Вторая модель лучше, тк r2 score выше

# *Задание 3
Вызовите документацию для класса RandomForestRegressor,
найдите информацию об атрибуте feature_importances_.
С помощью этого атрибута найдите сумму всех показателей важности,
установите, какие два признака показывают наибольшую важность.


In [21]:
help(RandomForestRegressor)

Help on class RandomForestRegressor in module sklearn.ensemble._forest:

class RandomForestRegressor(ForestRegressor)
 |  RandomForestRegressor(n_estimators=100, *, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)
 |  
 |  A random forest regressor.
 |  
 |  A random forest is a meta estimator that fits a number of classifying
 |  decision trees on various sub-samples of the dataset and uses averaging
 |  to improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is controlled with the `max_samples` parameter if
 |  `bootstrap=True` (default), otherwise the whole dataset is used to build
 |  each tree.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  Parameters
 |  ----------

In [34]:
importances = model.feature_importances_
forest_importances = pd.Series(importances, index=feature_names)
forest_importances.sort_values(), forest_importances.sum()

(CHAS       0.001236
 ZN         0.001543
 RAD        0.005281
 INDUS      0.007138
 TAX        0.011525
 B          0.012451
 NOX        0.014269
 AGE        0.014299
 PTRATIO    0.018081
 CRIM       0.031676
 DIS        0.063973
 RM         0.402682
 LSTAT      0.415847
 dtype: float64,
 1.0)

In [36]:
# the most important features are RM and LSTAT, sum is 1

# *Задание 4
В этом задании мы будем работать с датасетом, с которым мы уже знакомы по домашнему заданию по библиотеке Matplotlib, это датасет Credit Card Fraud Detection.Для этого датасета мы будем решать задачу классификации - будем определять,какие из транзакциции по кредитной карте являются мошенническими.Данный датасет сильно несбалансирован (так как случаи мошенничества относительно редки),так что применение метрики accuracy не принесет пользы и не поможет выбрать лучшую модель.Мы будем вычислять AUC, то есть площадь под кривой ROC.

Импортируйте из соответствующих модулей RandomForestClassifier, GridSearchCV и train_test_split.
Загрузите датасет creditcard.csv и создайте датафрейм df.


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

df = pd.read_csv('creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


С помощью метода value_counts с аргументом normalize=True убедитесь в том, что выборка несбалансирована. Используя метод info, проверьте, все ли столбцы содержат числовые данные и нет ли в них пропусков.Примените следующую настройку, чтобы можно было просматривать все столбцы датафрейма:
pd.options.display.max_columns = 100.

Просмотрите первые 10 строк датафрейма df.


In [46]:
pd.DataFrame(df.value_counts(normalize = True))

0
Time     V1         V2         V3        V4        V5        V6        V7         V8         V9        V10       V11       V12       V13       V14       V15       V16       V17       V18       V19       V20       V21        V22       V23       V24       V25       V26       V27       V28       Amount Class          
163152.0 -1.196037   1.585949   2.883976  3.378471  1.511706  3.717077  0.585362  -0.156001   0.122648  4.217934  1.385525 -0.709405 -0.256168 -1.564352  1.693218 -0.785210 -0.228008 -0.412833  0.234834  1.375790 -0.370294   0.524395 -0.355170 -0.869790 -0.133198  0.327804 -0.035702 -0.858197 7.56   0      0.000063
         -1.203617   1.574009   2.889277  3.381404  1.538663  3.698747  0.560211  -0.150911   0.124136  4.220998  1.384569 -0.706897 -0.256274 -1.562583  1.692915 -0.787338 -0.226776 -0.412354  0.234322  1.385597 -0.366727   0.522223 -0.357329 -0.870174 -0.134166  0.327019 -0.042648 -0.855262 1.51   0      0.000063
43153.0  -2.086016   2.203265   1.654339  2.941050 -1.683045  0.529728 -1.352162   1.793449  -0.723686  0.600365 -0.982212 -0.551636 -1.337000  0.834403  1.251862  0.033455  1.067978  0.160510  0.213087  0.079002  0.216444   0.567241 -0.035345  0.370201  0.157378  0.440341  0.210230  0.090558 0.76   0      0.000032
170731.0  2.033492   0.766969  -2.107555  3.631952  1.348594 -0.499907  0.945159  -0.286392  -1.370581  1.653073 -1.600434 -1.510901 -2.143280  1.189850 -0.875588  0.175808 -0.419433 -0.464717 -1.414528 -0.430560  0.241894   0.658545 -0.102644  0.580535  0.643637  0.347240 -0.116618 -0.078601 0.76   0      0.000032
68207.0  -13.192671  12.785971 -9.906650  3.320337 -4.801176  5.760059 -18.750889 -37.353443 -0.391540 -5.052502  4.406806 -4.610756 -1.909488 -9.072711 -0.226074 -6.211557 -6.248145 -3.149247  0.051576 -3.493050  27.202839 -8.887017  5.303607 -0.639435  0.263203 -0.108877  1.269566  0.939407 1.00   1      0.000021
...                                                                                                                                                                                                                                                                                                                      ...
65149.0  -0.608037   0.277482   2.333740  0.713876 -0.686327  0.424502  0.158410   0.277078   0.005665 -0.574444 -0.383596  0.063757  0.435809 -0.294166  1.561564  0.430549 -0.512260  0.321857 -1.089111  0.192164  0.425425   1.077523  0.095700  0.080007 -0.087784 -0.253436  0.077868  0.055774 115.98 0      0.000004
          0.890428  -0.914533   0.916273  0.533497 -1.417793 -0.283902 -0.520284   0.002223  -1.050330  0.827726  1.336306  0.961705  0.778165  0.101997  0.352339 -0.892199 -0.538873  1.792922 -1.092627 -0.119284 -0.239564  -0.634749 -0.018377  0.482486  0.102384 -0.559266  0.040121  0.067240 192.05 0      0.000004
65150.0  -0.819167   1.289630   1.155617 -0.356589  0.742668 -1.179886  1.114827  -0.105033  -1.169136 -1.218791  1.841286  0.558376  0.081792 -1.028918 -0.400824  0.722658  0.402985  0.613975 -0.605494 -0.014715 -0.011025  -0.125263 -0.385443  0.449483  0.536560  0.252429 -0.020876  0.072608 0.76   0      0.000004
         -0.283939   1.355339   0.553398  0.255501  0.561040 -1.338352  1.056880  -0.229176  -0.738105 -1.157676  0.000759 -0.543236 -0.313497 -1.370815  0.770587  0.452886  1.064176  0.458320 -0.199074 -0.019922 -0.076192  -0.211969 -0.256209  0.259185  0.096589  0.327896  0.021232  0.083294 0.76   0      0.000004
172792.0 -0.533413  -0.189733   0.703337 -0.506271 -0.012546 -0.649617  1.577006  -0.414650   0.486180 -0.915427 -1.040458 -0.031513 -0.188093 -0.084316  0.041333 -0.302620 -0.660377  0.167430 -0.256117  0.382948  0.261057   0.643078  0.376777  0.008797 -0.473649 -0.818267 -0.002415  0.013649 217.00 0      0.000004

[283726 rows x 1 columns]

In [42]:
pd.options.display.max_columns = 100
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

Создайте датафрейм X из датафрейма df, исключив столбец Class.\
Создайте объект Series под названием y из столбца Class.\
Разбейте X и y на тренировочный и тестовый наборы данных при помощи функции train_test_split, используя аргументы: test_size=0.3, random_state=100, stratify=y.\
У вас должны получиться объекты X_train, X_test, y_train и y_test.\
Просмотрите информацию о их форме.


In [48]:
X = df.drop(['Class'], axis=1)
X.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [50]:
y = pd.Series(df['Class'])
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Class, dtype: int64

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((199364, 30), (85443, 30), (199364,), (85443,))

Для поиска по сетке параметров задайте такие параметры:\
parameters = [{'n_estimators': [10, 15],\
'max_features': np.arange(3, 5),\
'max_depth': np.arange(4, 7)}]\
Создайте модель GridSearchCV со следующими аргументами:\
estimator=RandomForestClassifier(random_state=100),\
param_grid=parameters,\
scoring='roc_auc',\
cv=3.\
Обучите модель на тренировочном наборе данных (может занять несколько минут).


In [52]:
parameters = [{'n_estimators': [10, 15],
                'max_features': np.arange(3, 5),
                'max_depth': np.arange(4, 7)}]


gscv = GridSearchCV(estimator=RandomForestClassifier(random_state=100), 
                    param_grid=parameters, scoring='roc_auc', cv=3)
gscv.fit(X_train, y_train)

Просмотрите параметры лучшей модели с помощью атрибута best_params_.\
Предскажите вероятности классов с помощью полученнной модели и метода predict_proba.\
Из полученного результата (массив Numpy) выберите столбец с индексом 1 (вероятность класса 1) и запишите в массив y_pred_proba. Из модуля sklearn.metrics импортируйте метрику roc_auc_score.\
Вычислите AUC на тестовых данных и сравните с результатом,полученным на тренировочных данных, используя в качестве аргументов массивы y_test и y_pred_proba.


In [54]:
best_pr = gscv.best_params_
best_pr

{'max_depth': 6, 'max_features': 3, 'n_estimators': 15}

In [68]:
y_pred_cv = gscv.predict_proba(X_test)
y_pred_proba = y_pred_cv[:,1]

In [69]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred_proba)

0.9462664156037156